# Cleaning up
Deep Learning - KI29  
Deggendorf Institute of Technology  
Prof. Dr. Florian Wahl

## Modell Klasse einführen

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nnfs
from nnfs.datasets import sine_data

In [53]:
# Dense Layer
class Layer_Dense:

    # Initialization Code
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        # Initilalize weights and biases according to the shape given
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
        # Save Regularization Lambdas
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.inputs = inputs

        # Calculate output as we did on the slides
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        
        # Regularisation
        # L1 weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
            
        # L2 weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
            
        # L1 biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
            
        # L2 biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        # Gradients on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [54]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs

        # Calculate the output based on inputs.
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Copy before we modify
        self.dinputs = dvalues.copy()

        # Set to 0 if value is <=0
        self.dinputs[self.inputs <= 0] = 0

In [55]:
class Loss:
    def calculate(self, output, y):
        # Calculate the per sample loss
        samples_losses = self.forward(output, y)

        # Calculate the mean loss and return it
        loss = np.mean(samples_losses)
        return loss
    
    def regularization_loss(self, layer):
        # Init return value to 0
        regularization_loss = 0
        
        # L1 weights
        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
            
        # L2 weights
        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights ** 2)
            
        # L1 biases
        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
            
        # L2 biases
        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases ** 2)
            
        return regularization_loss

In [56]:
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        n_samples = len(y_pred)  # Count the samples

        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)  # Clip the predictions

        # Get correct confidence values
        # if labels are sparse
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(n_samples), y_true]

        # else if labels are one hot encoded
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Compute Losses
        losses = -np.log(correct_confidences)
        return losses

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)

        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues

        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [57]:
# Softmax activation
class Activation_Softmax:
    def forward(self, inputs):
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        # Normalize them for each sample
        self.output = probabilities

    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(
            zip(self.output, dvalues)
        ):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)

            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(
                single_output, single_output.T
            )

            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [58]:
# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):  # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded, # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [59]:
# SGD optimizer
class Optimizer_SGD:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    # Call before running optimization
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (
                1.0 / (1.0 + self.decay * self.iterations)
            )

    # Update parameters
    def update_params(self, layer):
        if self.momentum:
            # If layer has no momentum arrays, create them
            if not hasattr(layer, "weight_momentums"):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Compute weight updates
            weight_updates = (
                self.momentum * layer.weight_momentums
                - self.current_learning_rate * layer.dweights
            )
            layer.weight_momentums = weight_updates

            # Compute bias updates
            bias_updates = (
                self.momentum * layer.bias_momentums
                - self.current_learning_rate * layer.dbiases
            )
            layer.bias_momentums = bias_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        # Perform update
        layer.weights += weight_updates
        layer.biases += bias_updates

    # Call after running optimization
    def post_update_params(self):
        self.iterations += 1

In [60]:
class Optimizer_Adagrad:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1.0, decay=0.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    # Call before running optimization
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (
                1.0 / (1.0 + self.decay * self.iterations)
            )

    # Update parameters
    def update_params(self, layer):
        # If layer has no cache arrays, create them
        if not hasattr(layer, "weight_cache"):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update the cache
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2

        # Compute weight updates
        layer.weights += (
            -self.current_learning_rate
            * layer.dweights
            / (np.sqrt(layer.weight_cache) + self.epsilon)
        )

        # Compute bias updates
        layer.biases += (
            -self.current_learning_rate
            * layer.dbiases
            / (np.sqrt(layer.bias_cache) + self.epsilon)
        )

    # Call after running optimization
    def post_update_params(self):
        self.iterations += 1

In [61]:
class Optimizer_RMSprop:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1.0, decay=0.0, epsilon=1e-7, rho=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho

    # Call before running optimization
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (
                1.0 / (1.0 + self.decay * self.iterations)
            )

    # Update parameters
    def update_params(self, layer):
        # If layer has no cache arrays, create them
        if not hasattr(layer, "weight_cache"):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update the cache
        layer.weight_cache = (
            self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights**2
        )
        layer.bias_cache = (
            self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases**2
        )

        # Compute weight updates
        layer.weights += (
            -self.current_learning_rate
            * layer.dweights
            / (np.sqrt(layer.weight_cache) + self.epsilon)
        )

        # Compute bias updates
        layer.biases += (
            -self.current_learning_rate
            * layer.dbiases
            / (np.sqrt(layer.bias_cache) + self.epsilon)
        )

    # Call after running optimization
    def post_update_params(self):
        self.iterations += 1

In [62]:
class Optimizer_Adam:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=0.001, decay=0.0, epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # Call before running optimization
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (
                1.0 / (1.0 + self.decay * self.iterations)
            )

    # Update parameters
    def update_params(self, layer):
        # If layer has no cache arrays, create them
        if not hasattr(layer, "weight_cache"):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
            layer.bias_momentums = np.zeros_like(layer.biases)
            
        # Compute momentums
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases
        
        # Perform momentum correction using beta 1 (add +1 to avoid zero-division error)
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        # Update the cache
        layer.weight_cache = (
            self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        )
        layer.bias_cache = (
            self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2
        )
        
        # Perform cache correction
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Compute weight updates
        layer.weights += (
            -self.current_learning_rate
            * weight_momentums_corrected
            / (np.sqrt(weight_cache_corrected) + self.epsilon)
        )

        # Compute bias updates
        layer.biases += (
            -self.current_learning_rate
            * bias_momentums_corrected
            / (np.sqrt(bias_cache_corrected) + self.epsilon)
        )

    # Call after running optimization
    def post_update_params(self):
        self.iterations += 1

In [63]:
class Layer_Dropout:
    def __init__(self, rate):
        # Store the dropout probability which is 1-dropout_rate
        self.rate = 1 - rate
        
    def forward(self, inputs):
        # Save inputs
        self.inputs = inputs
        
        # Generate dropout mask
        self.binary_mask = np.random.binomial(1, self.rate, size=inputs.shape) / self.rate
        
        # Apply output mask
        self.output = inputs * self.binary_mask
        
    def backward(self, dvalues):
        # Apply gradient on values
        self.dinputs = dvalues * self.binary_mask

In [64]:
class Activation_Sigmoid:
    def forward(self, inputs):
        # Save inputs and calculate output
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))
        
    def backward(self, dvalues):
        # Calculate derivate (remember to use the "trick")
        self.dinputs = dvalues * (1 - self.output) * self.output

In [65]:
# Binary cross-entropy loss
class Loss_BinaryCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Calculate sample-wise loss
        sample_losses = -(y_true * np.log(y_pred_clipped) +
                          (1 - y_true) * np.log(1 - y_pred_clipped))
        sample_losses = np.mean(sample_losses, axis=-1)

        # Return losses
        return sample_losses

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of outputs in every sample
        # We'll use the first sample to count them
        outputs = len(dvalues[0])

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        clipped_dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)

        # Calculate gradient
        self.dinputs = -(y_true / clipped_dvalues -
                         (1 - y_true) / (1 - clipped_dvalues)) / outputs
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [66]:
# Dense Layer
class Layer_Dense:

    # Initialization Code
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        # Initilalize weights and biases according to the shape given
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
        # Save Regularization Lambdas
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.inputs = inputs

        # Calculate output as we did on the slides
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        
        # Regularisation
        # L1 weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
            
        # L2 weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
            
        # L1 biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
            
        # L2 biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        # Gradients on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [67]:
class Activation_Linear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs
        
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

In [68]:
class Loss_MeanSquaredError(Loss):
    def forward(self, y_pred, y_true):
        samples_losses = np.mean((y_true - y_pred)**2, axis=-1)
        return samples_losses
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        
        self.dinputs = -2 * (y_true - dvalues) / outputs
        
        self.dinputs = self.dinputs / samples

In [69]:
class Loss_MeanAbsoluteError(Loss):
    def forward(self, y_pred, y_true):
        sample_losses = np.mean(np.abs(y_true - y_pred), axis=-1)
        return sample_losses
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        
        self.dinputs = np.sign(y_true - dvalues) / outputs
        self.dinputs = self.dinputs / samples

## Input Layer

In [70]:
class Layer_Input:
    # Forward pass
    def forward(self, inputs):
        self.output = inputs

## Model Class Forward Pass

In [71]:
class Model:
    def __init__(self):
        self.layers = []
        
    def add(self, layer):
        self.layers.append(layer)
        
    def set(self, *, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer
        
    def finalize(self):
        # Create the input layer
        self.input_layer = Layer_Input()
        
        # Count layers
        layer_count = len(self.layers)
        
        # Iterate through and connect layers
        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]

            elif i < layer_count-1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]

            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                
    def forward(self, X):
        # Perform forward pass
        self.input_layer.forward(X)

        for layer in self.layers:
            layer.forward(layer.prev.output)

        return layer.output
        
    def train(self, X, y, *, epochs=1, print_every=1):
        for epoch in range(1, epochs+1):
            output = self.forward(X)
            print(output)
        

In [72]:
nnfs.init()

X, y = sine_data()

model = Model()
model.add(Layer_Dense(1, 64))
model.add(Activation_ReLU())
model.add(Layer_Dense(64, 64))
model.add(Activation_ReLU())
model.add(Layer_Dense(64, 1))
model.add(Activation_Linear())

model.set(
    loss=Loss_MeanSquaredError(),
    optimizer=Optimizer_Adam(learning_rate=0.005, decay=1e-3)
)

model.finalize()

output = model.train(X, y, epochs=2, print_every=100)
output[:1]

[[ 0.00000000e+00]
 [-1.13209162e-05]
 [-2.26418324e-05]
 [-3.39627550e-05]
 [-4.52836648e-05]
 [-5.66045856e-05]
 [-6.79255099e-05]
 [-7.92464198e-05]
 [-9.05673296e-05]
 [-1.01888261e-04]
 [-1.13209171e-04]
 [-1.24530066e-04]
 [-1.35851020e-04]
 [-1.47171915e-04]
 [-1.58492840e-04]
 [-1.69813764e-04]
 [-1.81134659e-04]
 [-1.92455554e-04]
 [-2.03776523e-04]
 [-2.15097418e-04]
 [-2.26418342e-04]
 [-2.37739223e-04]
 [-2.49060133e-04]
 [-2.60381144e-04]
 [-2.71702040e-04]
 [-2.83022935e-04]
 [-2.94343830e-04]
 [-3.05664696e-04]
 [-3.16985679e-04]
 [-3.28306574e-04]
 [-3.39627528e-04]
 [-3.50948452e-04]
 [-3.62269318e-04]
 [-3.73590243e-04]
 [-3.84911109e-04]
 [-3.96232092e-04]
 [-4.07553045e-04]
 [-4.18873911e-04]
 [-4.30194836e-04]
 [-4.41515760e-04]
 [-4.52836684e-04]
 [-4.64157580e-04]
 [-4.75478446e-04]
 [-4.86799399e-04]
 [-4.98120266e-04]
 [-5.09441248e-04]
 [-5.20762289e-04]
 [-5.32083155e-04]
 [-5.43404080e-04]
 [-5.54724946e-04]
 [-5.66045870e-04]
 [-5.77366678e-04]
 [-5.8868766

TypeError: 'NoneType' object is not subscriptable

## Model Class Backpropagation

In [73]:
class Model:
    def __init__(self):
        self.layers = []
        
    def add(self, layer):
        self.layers.append(layer)
        
    def set(self, *, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer
        
    def finalize(self):
        # Create the input layer
        self.input_layer = Layer_Input()
        
        # Count layers
        layer_count = len(self.layers)
        
        self.trainable_layers = []
        
        # Iterate through and connect layers
        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
                
            elif i < layer_count-1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
                
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss

            if hasattr(self.layers[i], "weights"):
                self.trainable_layers.append(self.layers[i])
                
    def forward(self, X):
        # Perform forward pass
        self.input_layer.forward(X)
        
        for layer in self.layers:
            layer.forward(layer.prev.output)
            
        return layer.output
    
    def train(self, X, y, *, epochs=1, print_every=1):
        for epoch in range(1, epochs+1):
            # Perform forward pass
            output = self.forward(X)
            print(output)

In [74]:
class Loss:
    def remember_trainable_layers(self, trainable_layers):
        self.trainable_layers = trainable_layers
    
    def calculate(self, output, y):
        # Calculate the per sample loss
        samples_losses = self.forward(output, y)
        
        # Compute data loss
        data_loss = np.mean(samples_losses)

        return data_loss, self.regularization_loss()
    
    def regularization_loss(self):
        # Init return value to 0
        regularization_loss = 0
        
        for layer in self.trainable_layers:
        
            # L1 weights
            if layer.weight_regularizer_l1 > 0:
                regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

            # L2 weights
            if layer.weight_regularizer_l2 > 0:
                regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights ** 2)

            # L1 biases
            if layer.bias_regularizer_l1 > 0:
                regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

            # L2 biases
            if layer.bias_regularizer_l2 > 0:
                regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases ** 2)
            
        return regularization_loss

In [75]:
class Loss_MeanSquaredError(Loss):
    def forward(self, y_pred, y_true):
        samples_losses = np.mean((y_true - y_pred)**2, axis=-1)
        return samples_losses
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        
        self.dinputs = -2 * (y_true - dvalues) / outputs
        
        self.dinputs = self.dinputs / samples

In [76]:
# Softmax activation
class Activation_Softmax:
    def forward(self, inputs):
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        # Normalize them for each sample
        self.output = probabilities

    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(
            zip(self.output, dvalues)
        ):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)

            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(
                single_output, single_output.T
            )

            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)
            
    # Calculate predictions for outputs
    def predictions(self, outputs): 
        return np.argmax(outputs, axis=1)

Add predictions method to Activation functions

In [77]:
# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):  # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded, # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples
    
    def predictions(self, outputs):
        return np.argmax(outputs, axis=1)

In [78]:
class Activation_Sigmoid:
    def forward(self, inputs):
        # Save inputs and calculate output
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))
        
    def backward(self, dvalues):
        # Calculate derivate (remember to use the "trick")
        self.dinputs = dvalues * (1 - self.output) * self.output
        
    def predictions(self, outputs):
        return (outputs > 0.5) * 1

In [79]:
class Activation_Linear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs
        
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        
    def predictions(self, outputs):
        return outputs

In [80]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs

        # Calculate the output based on inputs.
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Copy before we modify
        self.dinputs = dvalues.copy()

        # Set to 0 if value is <=0
        self.dinputs[self.inputs <= 0] = 0
        
    def predictions(self, outputs):
        return outputs

## Add Accuracy classes for regression and classification problems

In [81]:
class Accuracy:
    def calculate(self, predictions, y):
        comparisons = self.compare(predictions, y)
        accuracy = np.mean(comparisons)
        return accuracy

In [82]:
class Accuracy_Regression(Accuracy):
    def __init__(self):
        self.precision = None
        
    def init(self, y, reinit=False):
        if self.precision is None or reinit:
            self.precision = np.std(y) / 250
            
    def compare(self, predictions, y):
        return np.abs(predictions - y) < self.precision

In [83]:
class Model:
    def __init__(self):
        self.layers = []
        
    def add(self, layer):
        self.layers.append(layer)
        
    def set(self, *, loss, optimizer, accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuracy = accuracy
        
    def finalize(self):
        # Create the input layer
        self.input_layer = Layer_Input()
        
        # Count layers
        layer_count = len(self.layers)
        
        self.trainable_layers = []
        
        # Iterate through and connect layers
        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
                
            elif i < layer_count-1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
                
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                self.output_layer_activation = self.layers[i]
                
            if hasattr(self.layers[i], 'weights'):
                self.trainable_layers.append(self.layers[i])
                
        # Update loss with trainable layers
        self.loss.remember_trainable_layers(self.trainable_layers)
                
    def forward(self, X):
        # Perform forward pass
        self.input_layer.forward(X)
        
        for layer in self.layers:
            layer.forward(layer.prev.output)
            
        return layer.output
    
    def train(self, X, y, *, epochs=1, print_every=1):
        self.accuracy.init(y)
        
        for epoch in range(1, epochs+1):
            # Perform forward pass
            output = self.forward(X)

            # Calculate loss
            data_loss, regularization_loss = self.loss.calculate(output, y)
            loss = data_loss + regularization_loss
            
            # Get predictions and compute accuracy
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuracy.calculate(predictions, y)
            
            # Perform backward pass
            self.backward(output, y)
            
            # Run Optimizer
            self.optimizer.pre_update_params()
            for layer in self.trainable_layers:
                self.optimizer.update_params(layer)
            self.optimizer.post_update_params()
            
            # Print
            if not epoch % print_every:
                print(f"Epoch: {epoch}, Acc: {accuracy:.3f}, Loss: {loss:.6f}, Data Loss: {data_loss:.6f}, Reg Loss: {regularization_loss:.6f}, LR: {self.optimizer.current_learning_rate:.6f}")

            
    def backward(self, output, y):
        self.loss.backward(output, y)
        
        # Call backward methods for all layers in reverse
        for layer in reversed(self.layers):
            layer.backward(layer.next.dinputs)



        

In [84]:
# Binary cross-entropy loss
class Loss_BinaryCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Calculate sample-wise loss
        sample_losses = -(y_true * np.log(y_pred_clipped) +
                          (1 - y_true) * np.log(1 - y_pred_clipped))
        sample_losses = np.mean(sample_losses, axis=-1)

        # Return losses
        return sample_losses

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of outputs in every sample
        # We'll use the first sample to count them
        outputs = len(dvalues[0])

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        clipped_dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)

        # Calculate gradient
        self.dinputs = -(y_true / clipped_dvalues -
                         (1 - y_true) / (1 - clipped_dvalues)) / outputs
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [85]:
nnfs.init()

X, y = sine_data()

model = Model()
model.add(Layer_Dense(1, 64))
model.add(Activation_ReLU())
model.add(Layer_Dense(64, 64))
model.add(Activation_ReLU())
model.add(Layer_Dense(64, 1))
model.add(Activation_Linear())

model.set(
    loss=Loss_MeanSquaredError(),
    optimizer=Optimizer_Adam(learning_rate=0.005, decay=1e-3),
    accuracy=Accuracy_Regression()
)

model.finalize()

model.train(X, y, epochs=10000, print_every=100)

Epoch: 100, Acc: 0.017, Loss: 0.049211, Data Loss: 0.049211, Reg Loss: 0.000000, LR: 0.004550
Epoch: 200, Acc: 0.420, Loss: 0.000787, Data Loss: 0.000787, Reg Loss: 0.000000, LR: 0.004170
Epoch: 300, Acc: 0.787, Loss: 0.000066, Data Loss: 0.000066, Reg Loss: 0.000000, LR: 0.003849
Epoch: 400, Acc: 0.885, Loss: 0.000020, Data Loss: 0.000020, Reg Loss: 0.000000, LR: 0.003574
Epoch: 500, Acc: 0.053, Loss: 0.000121, Data Loss: 0.000121, Reg Loss: 0.000000, LR: 0.003336
Epoch: 600, Acc: 0.913, Loss: 0.000006, Data Loss: 0.000006, Reg Loss: 0.000000, LR: 0.003127
Epoch: 700, Acc: 0.922, Loss: 0.000004, Data Loss: 0.000004, Reg Loss: 0.000000, LR: 0.002943
Epoch: 800, Acc: 0.040, Loss: 0.000305, Data Loss: 0.000305, Reg Loss: 0.000000, LR: 0.002779
Epoch: 900, Acc: 0.932, Loss: 0.000003, Data Loss: 0.000003, Reg Loss: 0.000000, LR: 0.002633
Epoch: 1000, Acc: 0.936, Loss: 0.000002, Data Loss: 0.000002, Reg Loss: 0.000000, LR: 0.002501
Epoch: 1100, Acc: 0.685, Loss: 0.000007, Data Loss: 0.00000

Now let's make it work for classification

In [86]:
class Accuracy_Categorical(Accuracy):
    def __init__(self, *, binary=False):
        self.binary = binary
        
    def init(self, y):
        pass
    
    def compare(self, predictions, y):
        if not self.binary and len(y.shape) == 2:
            y = np.argmax(y, axis=1)
        return predictions == y

Let's add validation data

In [87]:
class Loss:
    def remember_trainable_layers(self, trainable_layers):
        self.trainable_layers = trainable_layers
    
    def calculate(self, output, y, *, include_regularization=False):
        # Calculate the per sample loss
        samples_losses = self.forward(output, y)
        
        # Compute data loss
        data_loss = np.mean(samples_losses)
        
        if not include_regularization:
            return data_loss
        
        return data_loss, self.regularization_loss()
    
    def regularization_loss(self):
        # Init return value to 0
        regularization_loss = 0
        
        for layer in self.trainable_layers:
        
            # L1 weights
            if layer.weight_regularizer_l1 > 0:
                regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

            # L2 weights
            if layer.weight_regularizer_l2 > 0:
                regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights ** 2)

            # L1 biases
            if layer.bias_regularizer_l1 > 0:
                regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

            # L2 biases
            if layer.bias_regularizer_l2 > 0:
                regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases ** 2)
            
        return regularization_loss

In [88]:
# Binary cross-entropy loss
class Loss_BinaryCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Calculate sample-wise loss
        sample_losses = -(y_true * np.log(y_pred_clipped) +
                          (1 - y_true) * np.log(1 - y_pred_clipped))
        sample_losses = np.mean(sample_losses, axis=-1)

        # Return losses
        return sample_losses

    # Backward pass
    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of outputs in every sample
        # We'll use the first sample to count them
        outputs = len(dvalues[0])

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        clipped_dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)

        # Calculate gradient
        self.dinputs = -(y_true / clipped_dvalues -
                         (1 - y_true) / (1 - clipped_dvalues)) / outputs
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [89]:
class Model:
    def __init__(self):
        self.layers = []
        
    def add(self, layer):
        self.layers.append(layer)
        
    def set(self, *, loss, optimizer, accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuracy = accuracy
        
    def finalize(self):
        # Create the input layer
        self.input_layer = Layer_Input()
        
        # Count layers
        layer_count = len(self.layers)
        
        self.trainable_layers = []
        
        # Iterate through and connect layers
        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
                
            elif i < layer_count-1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
                
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                self.output_layer_activation = self.layers[i]
                
            if hasattr(self.layers[i], 'weights'):
                self.trainable_layers.append(self.layers[i])
                
        # Update loss with trainable layers
        self.loss.remember_trainable_layers(self.trainable_layers)
                
    def forward(self, X):
        # Perform forward pass
        self.input_layer.forward(X)
        
        for layer in self.layers:
            layer.forward(layer.prev.output)
            
        return layer.output
    
    def train(self, X, y, *, epochs=1, print_every=1, validation_data=None):
        self.accuracy.init(y)
        
        for epoch in range(1, epochs+1):
            # Perform forward pass
            output = self.forward(X)

            # Calculate loss
            data_loss, regularization_loss = self.loss.calculate(output, y, 
                                                                 include_regularization=True)
            loss = data_loss + regularization_loss
            
            # Get predictions and compute accuracy
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuracy.calculate(predictions, y)
            
            # Perform backward pass
            self.backward(output, y)
            
            # Run Optimizer
            self.optimizer.pre_update_params()
            for layer in self.trainable_layers:
                self.optimizer.update_params(layer)
            self.optimizer.post_update_params()
            
            # Print
            if not epoch % print_every: 
                print(
                    f'epoch: {epoch}, ' +
                    f'acc: {accuracy:.3f}, ' +
                    f'loss: {loss:.3f} (' +
                    f'data_loss: {data_loss:.3f}, ' +
                    f'reg_loss: {regularization_loss:.3f}), ' + 
                    f'lr: {self.optimizer.current_learning_rate:.5f}'
                )
                
        # Compute validation performance if validation data exists
        if validation_data is not None:
            X_val, y_val = validation_data

            output = self.forward(X_val)
            
            loss = self.loss.calculate(output, y_val)
            
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuracy.calculate(predictions, y_val)
            
            print(f"Validation: acc: {accuracy:.3f}, loss: {loss:.3f}")
            
    def backward(self, output, y):
        self.loss.backward(output, y)
        
        # Call backward methods for all layers in reverse
        for layer in reversed(self.layers):
            layer.backward(layer.next.dinputs)

In [90]:
from nnfs.datasets import spiral_data

In [91]:
X, y = spiral_data(samples=100, classes=2)
X_test, y_test = spiral_data(samples=100, classes=2)

y = y.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

model = Model()

model.add(Layer_Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4))
model.add(Activation_ReLU())
model.add(Layer_Dense(64, 1))
model.add(Activation_Sigmoid())

model.set(
    loss=Loss_BinaryCrossentropy(),
    optimizer=Optimizer_Adam(decay=5e-7),
    accuracy=Accuracy_Categorical(binary=True)    
)

model.finalize()

model.train(X, y, validation_data=(X_test, y_test), epochs=10000, print_every=500)

epoch: 500, acc: 0.635, loss: 0.657 (data_loss: 0.654, reg_loss: 0.003), lr: 0.00100
epoch: 1000, acc: 0.735, loss: 0.597 (data_loss: 0.580, reg_loss: 0.017), lr: 0.00100
epoch: 1500, acc: 0.835, loss: 0.542 (data_loss: 0.509, reg_loss: 0.034), lr: 0.00100
epoch: 2000, acc: 0.835, loss: 0.494 (data_loss: 0.445, reg_loss: 0.049), lr: 0.00100
epoch: 2500, acc: 0.870, loss: 0.439 (data_loss: 0.374, reg_loss: 0.065), lr: 0.00100
epoch: 3000, acc: 0.915, loss: 0.397 (data_loss: 0.321, reg_loss: 0.076), lr: 0.00100
epoch: 3500, acc: 0.950, loss: 0.355 (data_loss: 0.267, reg_loss: 0.088), lr: 0.00100
epoch: 4000, acc: 0.955, loss: 0.325 (data_loss: 0.231, reg_loss: 0.093), lr: 0.00100
epoch: 4500, acc: 0.970, loss: 0.302 (data_loss: 0.207, reg_loss: 0.095), lr: 0.00100
epoch: 5000, acc: 0.970, loss: 0.283 (data_loss: 0.188, reg_loss: 0.095), lr: 0.00100
epoch: 5500, acc: 0.980, loss: 0.266 (data_loss: 0.172, reg_loss: 0.093), lr: 0.00100
epoch: 6000, acc: 0.980, loss: 0.251 (data_loss: 0.159,

Adapting dropout

In [92]:
class Layer_Dropout:
    def __init__(self, rate):
        # Store the dropout probability which is 1-dropout_rate
        self.rate = 1 - rate
        
    def forward(self, inputs, training):
        # Save inputs
        self.inputs = inputs
        
        # If we are not in Training, just return values
        if not training:
            self.output = inputs.copy()
            return
        
        # Generate dropout mask
        self.binary_mask = np.random.binomial(1, self.rate, size=inputs.shape) / self.rate
        
        # Apply output mask
        self.output = inputs * self.binary_mask
        
    def backward(self, dvalues):
        # Apply gradient on values
        self.dinputs = dvalues * self.binary_mask

In [93]:
class Model:
    def __init__(self):
        self.layers = []
        self.softmax_classifier_output = None
        
    def add(self, layer):
        self.layers.append(layer)
        
    def set(self, *, loss, optimizer, accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuracy = accuracy
        
    def finalize(self):
        # Create the input layer
        self.input_layer = Layer_Input()
        
        # Count layers
        layer_count = len(self.layers)
        
        self.trainable_layers = []
        
        # Iterate through and connect layers
        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
                
            elif i < layer_count-1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
                
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                self.output_layer_activation = self.layers[i]
                
            if hasattr(self.layers[i], 'weights'):
                self.trainable_layers.append(self.layers[i])
                
        # Update loss with trainable layers
        self.loss.remember_trainable_layers(self.trainable_layers)
        
        # If Softmax + Categorical Crossentropy use the combo function
        if isinstance(self.layers[-1], Activation_Softmax) and \
           isinstance(self.loss, Loss_CategoricalCrossentropy):
            self.softmax_classifier_output = Activation_Softmax_Loss_CategoricalCrossentropy()
                
    def forward(self, X, training):
        # Perform forward pass
        self.input_layer.forward(X, training)
        
        for layer in self.layers:
            layer.forward(layer.prev.output, training)
            
        return layer.output
    
    def train(self, X, y, *, epochs=1, print_every=1, validation_data=None):
        self.accuracy.init(y)
        
        for epoch in range(1, epochs+1):
            # Perform forward pass
            output = self.forward(X, training=True)

            # Calculate loss
            data_loss, regularization_loss = self.loss.calculate(output, y, 
                                                                 include_regularization=True)
            loss = data_loss + regularization_loss
            
            # Get predictions and compute accuracy
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuracy.calculate(predictions, y)
            
            # Perform backward pass
            self.backward(output, y)
            
            # Run Optimizer
            self.optimizer.pre_update_params()
            for layer in self.trainable_layers:
                self.optimizer.update_params(layer)
            self.optimizer.post_update_params()
            
            # Print
            if not epoch % print_every: 
                print(
                    f'epoch: {epoch}, ' +
                    f'acc: {accuracy:.3f}, ' +
                    f'loss: {loss:.3f} (' +
                    f'data_loss: {data_loss:.3f}, ' +
                    f'reg_loss: {regularization_loss:.3f}), ' + 
                    f'lr: {self.optimizer.current_learning_rate:.5f}'
                )
                
        # Compute validation performance if validation data exists
        if validation_data is not None:
            X_val, y_val = validation_data

            output = self.forward(X_val, training=False)
            
            loss = self.loss.calculate(output, y_val)
            
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuracy.calculate(predictions, y_val)
            
            print(f"Validation: acc: {accuracy:.3f}, loss: {loss:.3f}")
            
    def backward(self, output, y):
        # If softmax + categorical crossentropy
        if self.softmax_classifier_output is not None:
            self.softmax_classifier_output.backward(output, y)
            self.layers[-1].dinputs = self.softmax_classifier_output.dinputs
            for layer in reversed(self.layers[:-1]):
                layer.backward(layer.next.dinputs)
            return
        
        
        self.loss.backward(output, y)
        
        # Call backward methods for all layers in reverse
        for layer in reversed(self.layers):
            layer.backward(layer.next.dinputs)

In [94]:
class Layer_Input:
    # Forward pass
    def forward(self, inputs, training):
        self.output = inputs

In [95]:
# Dense Layer
class Layer_Dense:

    # Initialization Code
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        # Initilalize weights and biases according to the shape given
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
        # Save Regularization Lambdas
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs, training):
        self.inputs = inputs

        # Calculate output as we did on the slides
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        
        # Regularisation
        # L1 weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
            
        # L2 weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
            
        # L1 biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
            
        # L2 biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        # Gradients on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [96]:
class Activation_ReLU:
    def forward(self, inputs, training):
        self.inputs = inputs

        # Calculate the output based on inputs.
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Copy before we modify
        self.dinputs = dvalues.copy()

        # Set to 0 if value is <=0
        self.dinputs[self.inputs <= 0] = 0
        
    def predictions(self, outputs):
        return outputs

In [97]:
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        n_samples = len(y_pred)  # Count the samples

        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)  # Clip the predictions

        # Get correct confidence values
        # if labels are sparse
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(n_samples), y_true]

        # else if labels are one hot encoded
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Compute Losses
        losses = -np.log(correct_confidences)
        return losses

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)

        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues

        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [98]:
# Softmax activation
class Activation_Softmax:
    def forward(self, inputs, training):
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        # Normalize them for each sample
        self.output = probabilities

    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(
            zip(self.output, dvalues)
        ):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)

            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(
                single_output, single_output.T
            )

            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)
            
    # Calculate predictions for outputs
    def predictions(self, outputs): 
        return np.argmax(outputs, axis=1)

In [99]:
# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def backward(self, dvalues, y_true):  # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded, # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples


In [101]:
X, y = spiral_data(samples=1000, classes=3)
X_test, y_test = spiral_data(samples=100, classes=3)

model = Model()

model.add(Layer_Dense(2, 512, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4))
model.add(Activation_ReLU())
model.add(Layer_Dropout(0.1))
model.add(Layer_Dense(512, 3))
model.add(Activation_Softmax())

model.set(
    loss=Loss_CategoricalCrossentropy(),
    optimizer=Optimizer_Adam(learning_rate=0.05, decay=5e-5),
    accuracy=Accuracy_Categorical()    
)

model.finalize()

model.train(X, y, validation_data=(X_test, y_test), epochs=10000, print_every=500)

epoch: 500, acc: 0.815, loss: 0.536 (data_loss: 0.459, reg_loss: 0.077), lr: 0.04878
epoch: 1000, acc: 0.837, loss: 0.490 (data_loss: 0.423, reg_loss: 0.067), lr: 0.04762
epoch: 1500, acc: 0.837, loss: 0.498 (data_loss: 0.432, reg_loss: 0.065), lr: 0.04651
epoch: 2000, acc: 0.851, loss: 0.481 (data_loss: 0.420, reg_loss: 0.062), lr: 0.04546
epoch: 2500, acc: 0.850, loss: 0.463 (data_loss: 0.397, reg_loss: 0.066), lr: 0.04445
epoch: 3000, acc: 0.840, loss: 0.477 (data_loss: 0.420, reg_loss: 0.058), lr: 0.04348
epoch: 3500, acc: 0.851, loss: 0.462 (data_loss: 0.404, reg_loss: 0.058), lr: 0.04256
epoch: 4000, acc: 0.849, loss: 0.455 (data_loss: 0.399, reg_loss: 0.056), lr: 0.04167
epoch: 4500, acc: 0.850, loss: 0.433 (data_loss: 0.380, reg_loss: 0.053), lr: 0.04082
epoch: 5000, acc: 0.840, loss: 0.479 (data_loss: 0.427, reg_loss: 0.052), lr: 0.04000
epoch: 5500, acc: 0.859, loss: 0.422 (data_loss: 0.371, reg_loss: 0.051), lr: 0.03922
epoch: 6000, acc: 0.851, loss: 0.443 (data_loss: 0.393,